## BME i9400
## Fall 2025
### Linear Algebra for Machine Learning - Part II: Eigenanalysis and Dimensionality Reduction

### Today's Problem

You are analyzing brain MRI scans from an Alzheimer's disease study. Each scan contains 256×256×180 voxels (≈11.8 million measurements). Your dataset has 500 patients across different disease stages.

**The Challenge:**
1. How can you find the primary patterns of brain atrophy?
2. Can you reduce 11.8 million measurements to just a handful of meaningful features?
3. How much information is lost when you compress the data?

**Today's goal**: Master eigenanalysis, PCA, and SVD to extract meaningful patterns from high-dimensional biomedical data.

### Learning Objectives

By the end of this lecture, you will be able to:
1. **Compute** eigenvalues and eigenvectors and interpret their meaning
2. **Implement** PCA from scratch using eigendecomposition
3. **Apply** SVD for data compression and denoising
4. **Evaluate** how many components to retain for analysis
5. **Interpret** principal components in clinical context

### Mini-Lecture: From Covariance to Components (15 minutes)

#### Part 1: Eigenvalues and Eigenvectors

**Definition:**
$$\mathbf{A}\mathbf{v} = \lambda\mathbf{v}$$

- $\mathbf{v}$: eigenvector (direction that doesn't rotate)
- $\lambda$: eigenvalue (scaling factor)

**Geometric Intuition:**
- Eigenvectors are the "natural axes" of your data
- Eigenvalues tell you how much variance exists along each axis

**For Covariance Matrices:**
- Eigenvectors = Principal components (directions of maximum variance)
- Eigenvalues = Variance along each principal component
- Always orthogonal (perpendicular) for symmetric matrices

#### Part 2: Principal Component Analysis (PCA)

**Algorithm:**
1. Center data: $\mathbf{X}_{centered} = \mathbf{X} - \bar{\mathbf{X}}$
2. Compute covariance: $\mathbf{C} = \frac{1}{n-1}\mathbf{X}_{centered}^T\mathbf{X}_{centered}$
3. Eigendecomposition: $\mathbf{C} = \mathbf{V}\mathbf{\Lambda}\mathbf{V}^T$
4. Sort by eigenvalues (descending)
5. Project: $\mathbf{X}_{new} = \mathbf{X}_{centered}\mathbf{V}$

**Why PCA?**
- Reduces dimensionality while preserving variance
- Removes correlations between features
- Identifies hidden patterns

**Clinical Applications:**
- Gene expression: Cell type identification
- Neuroimaging: Disease progression patterns
- Proteomics: Biomarker discovery

#### Part 3: Singular Value Decomposition (SVD)

**Decomposition:**
$$\mathbf{X} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^T$$

- $\mathbf{U}$: Left singular vectors (sample patterns)
- $\mathbf{\Sigma}$: Singular values (importance)
- $\mathbf{V}$: Right singular vectors (feature patterns)

**Relationship to PCA:**
- Principal components = Right singular vectors ($\mathbf{V}$)
- Eigenvalues = $(\text{singular values})^2 / (n-1)$
- More numerically stable than eigendecomposition

**Low-rank Approximation:**
$$\mathbf{X} \approx \sum_{i=1}^k \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

Keep only top $k$ components for compression

### Hands-On Lab: Eigenanalysis and Dimensionality Reduction (40 minutes)

**Task:** Work through the following notebook cells, executing the code that has been provided for you, and filling in the code where indicated by `# YOUR CODE HERE`.

**Micro-Deliverable**
Submit your completed Jupyter notebook in your my-work folder by the end of class.

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style and random seed
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
import sklearn
print(f"Scikit-learn version: {sklearn.__version__}")

#### Task 1: Understanding Eigendecomposition (10 minutes)

In [ ]:
# Create a simple 2D dataset to visualize eigenvectors
np.random.seed(42)

# Generate correlated 2D data
mean = [0, 0]
cov = [[4, 2.8],
       [2.8, 3]]
data_2d = np.random.multivariate_normal(mean, cov, 300)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov)

# Sort by eigenvalues
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print("Covariance Matrix:")
print(cov)
print(f"\nEigenvalues: {eigenvalues}")
print(f"\nEigenvector 1 (PC1): {eigenvectors[:, 0]}")
print(f"Eigenvector 2 (PC2): {eigenvectors[:, 1]}")
print(f"\nOrthogonal? (dot product ≈ 0): {np.dot(eigenvectors[:, 0], eigenvectors[:, 1]):.6f}")

In [ ]:
# Visualize eigenvectors
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Original data with eigenvectors
axes[0].scatter(data_2d[:, 0], data_2d[:, 1], alpha=0.5, s=20)

# Plot eigenvectors scaled by sqrt(eigenvalues)
origin = [0, 0]
for i in range(2):
    axes[0].arrow(origin[0], origin[1],
                  eigenvectors[0, i] * np.sqrt(eigenvalues[i]) * 2,
                  eigenvectors[1, i] * np.sqrt(eigenvalues[i]) * 2,
                  head_width=0.2, head_length=0.2,
                  fc='red' if i == 0 else 'blue',
                  ec='red' if i == 0 else 'blue',
                  linewidth=2,
                  label=f'PC{i+1} (λ={eigenvalues[i]:.2f})')

axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].set_title('Original Data with Eigenvectors')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# Project data onto eigenvectors
data_projected = data_2d @ eigenvectors

axes[1].scatter(data_projected[:, 0], data_projected[:, 1], alpha=0.5, s=20)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title('Data in Principal Component Space')
axes[1].grid(True, alpha=0.3)
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Eigenvectors show the directions of maximum variance")
print("- PC1 captures the main trend in the data")
print("- PC2 captures the remaining variance orthogonal to PC1")
print("- In PC space, features are uncorrelated")

#### Task 2: Implementing PCA from Scratch (15 minutes)

In [ ]:
# Generate gene expression data
def generate_gene_expression(n_samples=100, n_genes=500, n_cell_types=3):
    """Generate synthetic gene expression data with distinct cell types"""
    np.random.seed(42)

    # Base expression
    X = np.random.randn(n_samples, n_genes) * 0.5

    # Add cell type-specific signatures
    samples_per_type = n_samples // n_cell_types
    labels = []

    for i in range(n_cell_types):
        start = i * samples_per_type
        end = min((i + 1) * samples_per_type, n_samples)

        # Cell type signature genes
        signature_genes = np.random.choice(n_genes, 50, replace=False)
        X[start:end, signature_genes] += np.random.randn() * 2 # What is happening in this line of code?

        labels.extend([f'Type_{i+1}'] * (end - start))

    # Add noise
    X += np.random.randn(n_samples, n_genes) * 0.3

    return X, labels[:n_samples]

# Generate data
X_genes, cell_types = generate_gene_expression(n_samples=150, n_genes=1000)
print(f"Gene expression matrix: {X_genes.shape}")
print(f"  {X_genes.shape[0]} cells × {X_genes.shape[1]} genes")

In [ ]:
# TODO 1: Implement PCA from scratch

def pca_from_scratch(X, n_components=None):
    """
    Implement PCA using eigendecomposition
    """
    # Step 1: Center the data (subtract out the mean)
    # YOUR CODE HERE
    X_mean = np.mean(X, axis=0)
    X_centered = ...

    # Step 2: Compute covariance matrix
    # YOUR CODE HERE
    n_samples = X.shape[0]
    cov_matrix = ...

    # Step 3: Eigendecomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Step 4: Sort by eigenvalues (descending)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Step 5: Select components
    if n_components is not None:
        eigenvectors = eigenvectors[:, :n_components]
        eigenvalues = eigenvalues[:n_components]

    # Step 6: Project data
    X_transformed = X_centered @ eigenvectors

    # Calculate explained variance ratio
    explained_variance_ratio = eigenvalues / np.sum(eigenvalues)

    return X_transformed, eigenvectors, eigenvalues, explained_variance_ratio, X_mean

# Apply our PCA implementation
X_pca_manual, components_manual, eigenvals_manual, var_ratio_manual, _ = pca_from_scratch(X_genes, n_components=10)

# Compare with sklearn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_std=False)  # Only center, don't scale
X_centered_sklearn = scaler.fit_transform(X_genes)

pca_sklearn = PCA(n_components=10)
X_pca_sklearn = pca_sklearn.fit_transform(X_centered_sklearn)

print("PCA Implementation Comparison:")
print(f"\nExplained variance (first 3 PCs):")
print(f"  Manual:  {var_ratio_manual[:3]}")
print(f"  Sklearn: {pca_sklearn.explained_variance_ratio_[:3]}")

# Check if results match (may have sign flip)
correlation_pc1 = np.abs(np.corrcoef(X_pca_manual[:, 0], X_pca_sklearn[:, 0])[0, 1])
print(f"\nPC1 correlation between implementations: {correlation_pc1:.4f}")
print("Implementation is correct!" if correlation_pc1 > 0.999 else "Check implementation")

In [ ]:
# Visualize PCA results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Scree plot
axes[0, 0].bar(range(1, 11), var_ratio_manual[:10] * 100)
axes[0, 0].set_xlabel('Principal Component')
axes[0, 0].set_ylabel('Explained Variance (%)')
axes[0, 0].set_title('Scree Plot')
axes[0, 0].set_xticks(range(1, 11))

# Cumulative variance
cumulative_var = np.cumsum(var_ratio_manual[:10])
axes[0, 1].plot(range(1, 11), cumulative_var * 100, 'o-')
axes[0, 1].axhline(y=90, color='r', linestyle='--', label='90% threshold')
axes[0, 1].set_xlabel('Number of Components')
axes[0, 1].set_ylabel('Cumulative Variance (%)')
axes[0, 1].set_title('Cumulative Explained Variance')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# PC1 vs PC2 scatter
colors = {'Type_1': 'red', 'Type_2': 'blue', 'Type_3': 'green'}
for cell_type in set(cell_types):
    mask = np.array(cell_types) == cell_type
    axes[1, 0].scatter(X_pca_manual[mask, 0], X_pca_manual[mask, 1],
                      label=cell_type, alpha=0.6, s=30, c=colors[cell_type])

axes[1, 0].set_xlabel(f'PC1 ({var_ratio_manual[0]:.1%} var)')
axes[1, 0].set_ylabel(f'PC2 ({var_ratio_manual[1]:.1%} var)')
axes[1, 0].set_title('Cell Type Clustering')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# PC2 vs PC3 scatter
for cell_type in set(cell_types):
    mask = np.array(cell_types) == cell_type
    axes[1, 1].scatter(X_pca_manual[mask, 1], X_pca_manual[mask, 2],
                      label=cell_type, alpha=0.6, s=30, c=colors[cell_type])

axes[1, 1].set_xlabel(f'PC2 ({var_ratio_manual[1]:.1%} var)')
axes[1, 1].set_ylabel(f'PC3 ({var_ratio_manual[2]:.1%} var)')
axes[1, 1].set_title('Alternative View')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

n_components_90 = np.argmax(cumulative_var >= 0.9) + 1
print(f"\nComponents needed for 90% variance: {n_components_90}")
print(f"Dimensionality reduction: {X_genes.shape[1]} → {n_components_90} features")
print(f"Compression ratio: {X_genes.shape[1] / n_components_90:.1f}:1")

#### Task 3: SVD for Medical Image Compression (15 minutes)

In [ ]:
# Generate synthetic brain scan data
def generate_brain_scans(n_scans=50, image_size=64):
    """Generate synthetic brain scan data with disease progression"""
    np.random.seed(42)

    # Create coordinate grid
    x, y = np.meshgrid(np.linspace(-1, 1, image_size),
                       np.linspace(-1, 1, image_size))

    # Define brain regions (simplified)
    # Central ventricles
    ventricles = np.exp(-(x**2 + y**2) / 0.1)

    # Cortical regions
    cortex = np.exp(-((x**2 + y**2) - 0.5)**2 / 0.1)

    # Subcortical structures
    subcortical = (np.exp(-((x-0.3)**2 + y**2) / 0.05) +
                   np.exp(-((x+0.3)**2 + y**2) / 0.05))

    scans = []
    disease_stages = []

    for i in range(n_scans):
        # Disease progression (0=healthy, 1=severe)
        stage = i / n_scans

        # Simulate atrophy: ventricles enlarge, cortex shrinks
        scan = (1 - 0.3 * stage) * cortex + \
               (1 + 0.5 * stage) * ventricles + \
               (1 - 0.2 * stage) * subcortical

        # Add noise
        scan += np.random.normal(0, 0.1, scan.shape)
        scan = np.clip(scan, 0, None)

        scans.append(scan.flatten())
        disease_stages.append(stage)

    return np.array(scans), np.array(disease_stages), image_size

# Generate data
X_scans, disease_stages, img_size = generate_brain_scans(n_scans=100)
print(f"Brain scan data: {X_scans.shape}")
print(f"  {X_scans.shape[0]} scans × {X_scans.shape[1]} pixels")

In [ ]:
# TODO 2: Perform SVD on brain scans

# Center the data
X_scans_mean = np.mean(X_scans, axis=0)
X_scans_centered = X_scans - X_scans_mean

# YOUR CODE HERE: Perform SVD
# Hint: use np.linalg.svd with full_matrices=False
U, S, Vt = ...

print("SVD Decomposition:")
print(f"  U (scan coefficients): {U.shape}")
print(f"  S (singular values): {S.shape}")
print(f"  Vt (image components): {Vt.shape}")

# Calculate variance explained
variance_explained = (S**2) / np.sum(S**2)
cumulative_variance_svd = np.cumsum(variance_explained)

# Visualize singular value decay
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Singular values
axes[0].semilogy(S[:30], 'o-')
axes[0].set_xlabel('Component')
axes[0].set_ylabel('Singular Value')
axes[0].set_title('Singular Value Spectrum')
axes[0].grid(True, alpha=0.3)

# Variance per component
axes[1].bar(range(1, 16), variance_explained[:15] * 100)
axes[1].set_xlabel('Component')
axes[1].set_ylabel('Variance Explained (%)')
axes[1].set_title('Individual Component Variance')

# Cumulative variance
axes[2].plot(range(1, 31), cumulative_variance_svd[:30] * 100, 'o-')
axes[2].axhline(y=90, color='r', linestyle='--', label='90%')
axes[2].axhline(y=95, color='orange', linestyle='--', label='95%')
axes[2].axhline(y=99, color='green', linestyle='--', label='99%')
axes[2].set_xlabel('Number of Components')
axes[2].set_ylabel('Cumulative Variance (%)')
axes[2].set_title('Cumulative Variance')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Report compression potential
for threshold in [0.90, 0.95, 0.99]:
    n_comp = np.argmax(cumulative_variance_svd >= threshold) + 1
    compression = X_scans.shape[1] / n_comp
    print(f"{threshold:.0%} variance: {n_comp:3d} components, {compression:5.1f}:1 compression")

In [ ]:
# Visualize principal brain components ("eigenbrains")
fig, axes = plt.subplots(3, 4, figsize=(12, 9))

# Show mean brain
axes[0, 0].imshow(X_scans_mean.reshape(img_size, img_size), cmap='gray')
axes[0, 0].set_title('Mean Brain')
axes[0, 0].axis('off')

# Show first 11 components
for i in range(11):
    row = (i + 1) // 4
    col = (i + 1) % 4

    component = Vt[i].reshape(img_size, img_size)
    axes[row, col].imshow(component, cmap='RdBu_r')
    axes[row, col].set_title(f'PC{i+1} ({variance_explained[i]:.1%})')
    axes[row, col].axis('off')

plt.suptitle('Principal Components of Brain Scans', fontsize=14)
plt.tight_layout()
plt.show()

print("\nInterpretation of Components:")
print("  PC1: Overall brain size/intensity variation")
print("  PC2-3: Ventricular enlargement (atrophy pattern)")
print("  PC4-5: Cortical thickness variations")
print("  Later PCs: Finer anatomical details and noise")

In [ ]:
# TODO 3: Image reconstruction with different numbers of components

# Select test scans at different disease stages
test_indices = [0, 25, 50, 75, 99]  # Healthy to severe
n_components_list = [1, 5, 10, 20, 50]

fig, axes = plt.subplots(len(test_indices), len(n_components_list) + 1,
                        figsize=(15, 12))

for i, idx in enumerate(test_indices):
    # Original scan
    original = X_scans[idx].reshape(img_size, img_size)
    axes[i, 0].imshow(original, cmap='gray', vmin=0, vmax=2)
    axes[i, 0].set_title(f'Original\nStage: {disease_stages[idx]:.1f}')
    axes[i, 0].axis('off')

    # Reconstructions
    for j, n_comp in enumerate(n_components_list):
        # YOUR CODE HERE: Reconstruct using n_comp components
        X_reconstructed = ...
        X_reconstructed += X_scans_mean

        reconstructed = X_reconstructed[idx].reshape(img_size, img_size)

        axes[i, j+1].imshow(reconstructed, cmap='gray', vmin=0, vmax=2)
        axes[i, j+1].set_title(f'{n_comp} PCs\n({cumulative_variance_svd[n_comp-1]:.1%})')
        axes[i, j+1].axis('off')

plt.suptitle('Brain Scan Reconstruction Across Disease Stages', fontsize=14)
plt.tight_layout()
plt.show()

# Calculate reconstruction errors
print("Reconstruction Mean Squared Error:")
print("Components:", end="")
for n_comp in n_components_list:
    print(f"{n_comp:8d}", end="")
print()

for idx in test_indices:
    print(f"Stage {disease_stages[idx]:.1f}: ", end="")
    for n_comp in n_components_list:
        X_reconstructed = U[:, :n_comp] @ np.diag(S[:n_comp]) @ Vt[:n_comp, :]
        error = np.mean((X_scans_centered[idx] - X_reconstructed[idx])**2)
        print(f"{error:8.4f}", end="")
    print()

### Summary

**Key Takeaways:**
- Eigenanalysis reveals the natural structure of data
- PCA reduces dimensionality while preserving variance
- SVD provides stable, efficient matrix factorization
- 90% variance often captured with 10-20% of original dimensions
- Principal components often have biological interpretation

**Clinical Applications:**
- Disease subtype discovery
- Image compression and denoising
- Biomarker panel reduction
- Patient stratification